In [33]:
# import modules
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
import plotly.graph_objects as go

In [34]:


# load data
class RawDataLoader():
    def __init__(self, path='BMED_train_data_v2.xlsx'):
        self.path = path
        self.X_data, self.Y_data = self.RawData()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def RawData(self):
        df = pd.read_excel(self.path, sheet_name='Sheet2')
        X_data = df[['T','V','E','CF_LA','CF_K','CA_LA','CB_K']].values
        Y_data = df[['dNLA','dNK','dVF','dVA','dVB']].values
        return X_data, Y_data
    
    def PrepareData(self, test_size=0.2, random_state=42):
        # Split the data into training and test sets
        X_train, X_test, Y_train, Y_test = train_test_split(
            self.X_data, self.Y_data, 
            test_size=test_size, 
            random_state=random_state
            )
        
        # Normalize the data
        scaler_X = StandardScaler()
        scaler_Y = StandardScaler()

        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        Y_train_scaled = scaler_Y.fit_transform(Y_train)
        Y_test_scaled = scaler_Y.transform(Y_test)

        # Convert to PyTorch tensors and move to appropriate device
        X_train_tensor = torch.FloatTensor(X_train_scaled).to(self.device)
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(self.device)
        Y_train_tensor = torch.FloatTensor(Y_train_scaled).to(self.device)
        Y_test_tensor = torch.FloatTensor(Y_test_scaled).to(self.device)

        return X_train_tensor, X_test_tensor, Y_train_tensor, Y_test_tensor, scaler_X, scaler_Y
    
# Customize the NN architecture
class CustomModel(nn.Module):
    def __init__(self, hidden_layers=4, hidden_nodes = 128):
        super().__init__()
        layers = []
        nodes = 7
        for _ in range(hidden_layers):
            layers.append(nn.Linear(nodes, hidden_nodes))
            layers.append(nn.ReLU())
            nodes = hidden_nodes
        layers.append(nn.Linear(hidden_nodes, 5))
        self.hidden = nn.Sequential(*layers)

    def forward(self, x):
        return self.hidden(x) 

# Set the hyperparameters
class NNmodel():
    def __init__(self, hidden_layers=4, hidden_nodes = 128, learning_rate=0.0005791697935397798, num_epochs=9680, batch_size=16, weight_decay=1.0000000000000004e-06, title='bmed_flux_batch_NN_model.pth'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.hidden_layers = hidden_layers
        self.hidden_nodes = hidden_nodes
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.weight_decay = weight_decay
        self.model = CustomModel(hidden_layers=self.hidden_layers, hidden_nodes=self.hidden_nodes).to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        self.train_losses = []  # 각 에폭의 학습 손실 저장
        self.test_losses = []   # 각 에폭의 검증 손실 저장
        self.predictions = {}   # 예측 결과 저장
        self.title = title
        self.scaler_X = None
        self.scaler_Y = None

        if torch.cuda.is_available():
            print(f'Using GPU: {torch.cuda.get_device_name(0)}')
            print(f'Memory Usage: {torch.cuda.memory_allocated(0)/1024**2:.2f}MB')
        

    def train(self, X_train, Y_train, X_test, Y_test):
        try:
            X_train_gpu = X_train
            Y_train_gpu = Y_train
            X_test_gpu = X_test
            Y_test_gpu = Y_test

            dataset = TensorDataset(X_train_gpu, Y_train_gpu)
            dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
            
            for epoch in range(self.num_epochs):
                self.model.train()
                epoch_loss = 0
                batch_count = 0

                for X_batch, Y_batch in dataloader:
                    X_batch = X_batch.to(self.device)
                    Y_batch = Y_batch.to(self.device)
                    self.optimizer.zero_grad()
                    train_outputs = self.model(X_batch)
                    train_loss = self.criterion(train_outputs, Y_batch)
                    train_loss.backward()
                    self.optimizer.step()

                    epoch_loss += train_loss.item()
                    batch_count += 1

                # 각 에폭마다 학습 및 검증 손실 저장
                with torch.no_grad():
                    train_pred = self.model(X_train_gpu)
                    test_pred = self.model(X_test_gpu)
                    train_loss = self.criterion(train_pred, Y_train_gpu).item()
                    test_loss = self.criterion(test_pred, Y_test_gpu).item()
                    self.train_losses.append(train_loss)
                    self.test_losses.append(test_loss)

                if (epoch + 1) % 10 == 0:
                    print(f'\rEpoch [{epoch}/{self.num_epochs}] Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}', end='',flush=True)
                
                # 주기적으로 메모리 정리
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            
            # 최종 예측 결과 저장
            with torch.no_grad():
                test_outputs = self.model(X_test_gpu)
                self.predictions = {
                    'true': Y_test_gpu.cpu().numpy(),
                    'pred': test_outputs.cpu().numpy()
                }

            # 학습 곡선 시각화
            self.plot_learning_curves()
            # 예측 결과 시각화
            self.plot_predictions()

            # Save the model
            self.save_model(self.title)

            return test_loss.item()
        
        except RuntimeError as e:
            print(f"GPU 메모리 에러: {e}")
            # 메모리 해제
            del X_train_gpu, Y_train_gpu
            torch.cuda.empty_cache()
            raise
            
        finally:
            # 학습 완료 후 메모리 정리
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def save_model(self, title):
        model_state = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'hyperparameters': {
                'train_losses': self.train_losses,
                'test_losses': self.test_losses,
                'hidden_layers': self.hidden_layers,
                'hidden_nodes': self.hidden_nodes,
                'learning_rate': self.learning_rate,
                'num_epochs': self.num_epochs,
                'batch_size': self.batch_size,
                'weight_decay': self.weight_decay
            },
            'scalers': {
                'scaler_X': self.scaler_X,
                'scaler_Y': self.scaler_Y
            },
            'predictions': self.predictions,
            'train_losses': self.train_losses,
            'test_losses': self.test_losses
        }
        torch.save(model_state, title)
        print(f"Model saved to {title}")            
        
    def load_model(self, filepath):
        model_state = torch.load(filepath, weights_only=False)

        # load hyperparameters
        hyperparameters = model_state['hyperparameters']
        self.hidden_layers = hyperparameters['hidden_layers']
        self.hidden_nodes = hyperparameters['hidden_nodes']
        self.learning_rate = hyperparameters['learning_rate']
        self.num_epochs = hyperparameters['num_epochs']
        self.batch_size = hyperparameters['batch_size']
        self.weight_decay = hyperparameters['weight_decay']

        # load connection weights of the model
        self.model.load_state_dict(model_state['model_state_dict'])
        self.optimizer.load_state_dict(model_state['optimizer_state_dict'])
        self.train_losses = model_state['train_losses']
        self.test_losses = model_state['test_losses']
        self.predictions = model_state['predictions']

        # load scalers
        self.scaler_X = model_state['scalers']['scaler_X']
        self.scaler_Y = model_state['scalers']['scaler_Y']

        print(f"Model loaded from {filepath}")
        
    
    def plot_learning_curves(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.train_losses, label='Train Loss')
        plt.plot(self.test_losses, label='Test Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Learning Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig('learning_curves.png')
        plt.close()

    def plot_predictions(self, X_train, Y_train, X_test, Y_test):
        with torch.no_grad():
            # 학습 데이터 예측
            train_outputs = self.model(X_train)
            Y_train_true = Y_train.cpu().numpy()
            Y_train_pred = train_outputs.cpu().numpy()
            
            # 테스트 데이터 예측
            test_outputs = self.model(X_test)
            Y_test_true = Y_test.cpu().numpy()
            Y_test_pred = test_outputs.cpu().numpy()
            
            # 전체 데이터 합치기
            Y_all_true = np.vstack([Y_train_true, Y_test_true])
            Y_all_pred = np.vstack([Y_train_pred, Y_test_pred])
            
            # 스케일 복원
            Y_train_true = self.scaler_Y.inverse_transform(Y_train_true)
            Y_train_pred = self.scaler_Y.inverse_transform(Y_train_pred)
            Y_test_true = self.scaler_Y.inverse_transform(Y_test_true)
            Y_test_pred = self.scaler_Y.inverse_transform(Y_test_pred)
            Y_all_true = self.scaler_Y.inverse_transform(Y_all_true)
            Y_all_pred = self.scaler_Y.inverse_transform(Y_all_pred)
            
            output_names = ['dNLA', 'dNK', 'dVF', 'dVA', 'dVB']
            
            for i, name in enumerate(output_names):
                # 테스트 데이터 플롯
                plt.figure(figsize=(10, 6))
                r2_test = r2_score(Y_test_true[:, i], Y_test_pred[:, i])
                plt.scatter(Y_test_true[:, i], Y_test_pred[:, i], alpha=0.5, label=f'Test Data (R² = {r2_test:.3f})')
                
                min_val = min(min(Y_test_true[:, i]), min(Y_test_pred[:, i]))
                max_val = max(max(Y_test_true[:, i]), max(Y_test_pred[:, i]))
                plt.plot([min_val, max_val], [min_val, max_val], 'r--')
                
                plt.xlabel(f'Actual {name}')
                plt.ylabel(f'Predicted {name}')
                plt.title(f'Prediction vs Actual for {name} (Test Data)')
                plt.legend()
                plt.grid(True)
                plt.savefig(f'prediction_plot_{name}_test.png')
                plt.close()
                
                # 전체 데이터 플롯
                plt.figure(figsize=(10, 6))
                r2_all = r2_score(Y_all_true[:, i], Y_all_pred[:, i])
                plt.scatter(Y_all_true[:, i], Y_all_pred[:, i], alpha=0.5, label=f'All Data (R² = {r2_all:.3f})')
                
                min_val = min(min(Y_all_true[:, i]), min(Y_all_pred[:, i]))
                max_val = max(max(Y_all_true[:, i]), max(Y_all_pred[:, i]))
                plt.plot([min_val, max_val], [min_val, max_val], 'r--')
                
                plt.xlabel(f'Actual {name}')
                plt.ylabel(f'Predicted {name}')
                plt.title(f'Prediction vs Actual for {name} (All Data)')
                plt.legend()
                plt.grid(True)
                plt.savefig(f'prediction_plot_{name}_all.png')
                plt.close()

    def __del__(self):
        # 객체가 삭제될 때 메모리 정리
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

if __name__ == "__main__":
    # 데이터 로드
    data_loader = RawDataLoader()
    X_train, X_test, Y_train, Y_test, scaler_X, scaler_Y = data_loader.PrepareData()

    NN = NNmodel()
    NN.load_model('bmed_flux_batch_NN_model_v0.pth')
    NN.plot_learning_curves()
    NN.plot_predictions(X_train, Y_train, X_test, Y_test)

Using GPU: NVIDIA GeForce RTX 4080 SUPER
Memory Usage: 9.59MB
Model loaded from bmed_flux_batch_NN_model_v0.pth


In [35]:
import optuna
import sqlite3

# study.db에서 study 이름 확인
conn = sqlite3.connect('study.db')
cursor = conn.cursor()

# studies 테이블에서 study_name 조회
cursor.execute("SELECT study_name FROM studies")
study_names = cursor.fetchall()

print("Available studies:")
for name in study_names:
    print(f"- {name[0]}")

conn.close()

# 확인된 study 이름으로 로드
study = optuna.load_study(
    study_name="optimization_study",  # 위에서 확인된 이름을 입력
    storage="sqlite:///study.db"
)

# 최적화 히스토리 시각화
optuna.visualization.plot_optimization_history(study)

Available studies:
- optimization_study


In [36]:
X_train

tensor([[-0.8624,  0.8908, -0.2284,  ..., -0.9063, -0.5821, -0.7165],
        [-0.6940,  0.8908, -0.2284,  ..., -0.6657, -0.1427, -0.2059],
        [-1.4732, -0.2161, -0.2284,  ..., -0.6980, -0.4307, -0.1692],
        ...,
        [-0.8251, -1.3229, -0.2284,  ..., -0.4497, -0.6085, -0.5903],
        [ 0.6169,  0.8908, -0.2284,  ...,  1.1754, -0.2422,  0.7441],
        [-0.5728,  0.8908, -0.2284,  ..., -0.7751,  0.0939, -0.0619]],
       device='cuda:0')